# Functional model

In [611]:
#TODO: printing methods
#TODO: use Enums (?)
#TODO function to link parameters across populations (generate only once)
#TODO to_dataframeobj()
#TODO use recalculate_size ?

In [625]:
# Simulations study parameters
FMIN = 6 #minimum possible visual score
SMIN = 0 #minimum possible symptom score
NDAYS = 160 #number of days in the study
FIRSTVISIT = 8
LASTVISIT = NDAYS-1

from copy import copy
from warnings import warn
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib import colors as mcolors
from numpy import random
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000)
random.seed(3) # chosen by fair dice roll. guaranteed to be random. https://xkcd.com/221/

In [613]:
#helper functions

def uniform_to_normal(interval, pmsigma=3):
    '''
    Converts parameters for a uniform distribution to parameters for a normal distribution 

            Input:
                    interval (tuple of (float, float)): the interval boundaries (min and max)
                    pmsignma (float): half the standard deviations that the interval should comprise of
            Returns:
                    normal_params (tuple of (float, float)): mean and stdev
    '''
    mean = (interval[0]+interval[1])/2
    stdev = (interval[1]-interval[0])/(2*pmsigma)
    return (mean, stdev)

In [686]:
from types import MethodType

class Population:
    def __init__(self, size=10000, title=''):
        
        self.title = title
        self.size = size
        
        self.parameter_generators = {}
        self.parameters = {}
        self.function_generators = {}
        
        self.visualscores = None
        self.symptomscores_noerror = None
        self.symptomscores = None
        
    #Data generation methods

    def set_parameter_generator(self, paramname, func, paramtype):
        '''
        func must be a numpy function with the "shape" argument of the numpy function as the only argument to func
        paramtype must be either "population", "person", or "day"
        '''
        if func.__code__.co_varnames != ('shape',):
            raise ValueError("The function for parameter generation should only have 'shape' as an argument. "+
                             "Currently, it's arguments are: "
                            +", ".join(["'"+str(arg)+"'" for arg in func.__code__.co_varnames]))
        
        if paramtype == 'population':
            shape = (1,)
        elif paramtype == 'person':
            shape = (self.size, 1)
        elif paramtype == 'day':
            shape = (self.size, NDAYS)
        self.parameter_generators[paramname] = lambda: func(shape)
    def generate_parameters(self):
        '''generates all parameters
        if population was already generated, is resets the scores (since they were based on previous parameters)
        '''
        #generate
        for paramname in self.parameter_generators.keys():
            self.parameters[paramname] = self.parameter_generators[paramname]()
        #reset
        self.visualscores = None
        self.symptomscores_noerror = None
        self.symptomscores = None        
    
    def set_score_generator(self, scorename, func):
        '''scorename is either visualscore, symptomscore_noerror, or symptomscore'''
        self.function_generators[scorename] = func
    def generate_from_score_generator(self, generatorname):
        '''generatorname is either visualscore, symptomscore_noerror, or error'''
        func = self.function_generators[generatorname]
        paramnames = func.__code__.co_varnames
        paramvals = []
        for paramname in paramnames:
            #reserved parameters
            if paramname == 't':
                paramvals.append(np.arange(NDAYS))
            elif paramname == 'f':
                paramvals.append(self.visualscores)
            elif paramname == 's':
                paramvals.append(self.symptomscores_noerror)
            #custom parameters
            else:
                paramvals.append(self.parameters[paramname])
        return func(*paramvals)
    
    def generate(self, generate_parameters=True):
        if generate_parameters: self.generate_parameters()
        
        self.visualscores = self.generate_from_score_generator('visualscore')
        minscore = np.min(self.visualscores)
        if minscore < FMIN: 
            warn("visual score in {} has min={}, which is below FMIN={}".format(self.title, minscore, FMIN))
        self.symptomscores_noerror = self.generate_from_score_generator('symptomscore_noerror')
        self.symptomscores = self.generate_from_score_generator('symptomscore')
        minscore = np.min(self.symptomscores)
        if minscore < SMIN: 
            warn("symptom score in {} has  min={}, which is below SMIN={}".format(self.title, minscore, SMIN))
            
        #if all parameters are 'population', the generation process will only have created a single row
        #so, repeat that row 'self.size' times to create the full matrix
        if self.visualscores.shape != (self.size, NDAYS):
            self.visualscores = np.broadcast_to(self.visualscores, (self.size, NDAYS))
        if self.symptomscores_noerror.shape != (self.size, NDAYS):
            self.symptomscores_noerror = np.broadcast_to(self.symptomscores_noerror, (self.size, NDAYS))
        if self.symptomscores.shape != (self.size, NDAYS):
            self.symptomscores = np.broadcast_to(self.symptomscores, (self.size, NDAYS))
    
    #Non-data-generation methods
    
    def recalculate_size(self):
        #TODO try-catch for if not generated yet
        self.size = self.visualscores.shape[0]
        
    def copy(self, newtitle=None, additionaltitle='(copy)'):
        newpop = copy(self) #python shallow_copy
        if newtitle is not None:
            newpop.title = newtitle
        if additionaltitle is not None:
            newpop.title += ' '+additionaltitle
        newpop.parameter_generators = copy(self.parameter_generators)
        newpop.function_generators = copy(self.function_generators)
        return newpop
    
    #removing outliers #TODO should this be changing the population or creating a copy?
    def filter(self, recovered_symptom_score=SMIN, firstday=FIRSTVISIT, lastday=NDAYS):
        persons_recovered_early = np.any(self.symptomscores[:,:firstday] <= recovered_symptom_score, axis=1)
        persons_recovered_late = np.min(self.symptomscores[:,:lastday], axis=1) > recovered_symptom_score
        persons_excluded = np.logical_or(persons_recovered_early, persons_recovered_late)
        persons_included = np.logical_not(persons_excluded)

        self.visualscores = self.visualscores[persons_included]
        self.symptomscores_noerror = self.symptomscores_noerror[persons_included]
        self.symptomscores = self.symptomscores[persons_included]
        self.size = self.visualscores.shape[0]
    
    #plotting
    def plot(self, ax, ndays=NDAYS, npersons=None, score='symptom', viztype='lines'):
        #score is either 'symptom', 'visual', or 'symptom no error'
        #viztype is either 'lines', 'points', or 'both'
        
        if npersons is None: npersons=self.size
        
        if score=='visual':
            scores = self.visualscores[:npersons, :ndays]
            ax.set(title=self.title, xlabel='days since concussion', ylabel='visual score')
        if score=='symptom no error':
            scores = self.symptomscores_noerror[:npersons, :ndays]
            ax.set(title=self.title, xlabel='days since concussion', ylabel='symptom score without error')
        if score=='symptom':
            scores = self.symptomscores[:npersons, :ndays]
            ax.set(title=self.title, xlabel='days since concussion', ylabel='symptom score')
        
        colors = [mcolors.to_rgba(c) for c in plt.rcParams['axes.prop_cycle'].by_key()['color']] #list of 10 rgba colors
        if viztype=='lines' or viztype=='both':
            days = np.tile(np.arange(ndays), npersons).reshape((npersons, ndays))
            points = np.stack([days, scores], axis=2)
            ax.add_collection(LineCollection(points, colors=colors))
            ax.autoscale()
        if viztype=='points' or viztype=='both':
            days = np.tile(np.arange(ndays), npersons)
            edgecolors = np.repeat(colors[:npersons], ndays, axis=0)
            ax.scatter(days, scores.flatten(), facecolors='none', edgecolors=edgecolors)
            ax.autoscale()

# Population generation

In [683]:
# Various functions to create populations

def get_default_pop(size=10000):
    pop = Population(size, 'default pop')        

    #visual score
    gen_visualscores = lambda t,r,f0: np.maximum(-r*t+f0, FMIN)
    pop.set_score_generator('visualscore', gen_visualscores)
    gen_r = lambda shape: 1
    pop.set_parameter_generator('r', gen_r , 'person')
    gen_f0 = lambda shape: 16
    pop.set_parameter_generator('f0', gen_f0, 'person')

    #symptom score no error
    gen_symptomscores = lambda f,a: a*(f-FMIN)
    pop.set_score_generator('symptomscore_noerror', gen_symptomscores)
    gen_a = lambda shape: 1
    pop.set_parameter_generator('a', gen_a, 'population')

    #symptom error
    gen_error_mult = lambda s,C: s*C
    pop.set_score_generator('symptomscore', gen_error_mult)
    gen_C = lambda shape: 1
    pop.set_parameter_generator('C', gen_C, 'day')

    return pop

def get_poster_pop(size=10000):
    pop = Population(size, 'poster pop')        

    #visual score
    gen_visualscores = lambda t,r,f0: np.maximum(-r*t+f0, FMIN)
    pop.set_score_generator('visualscore', gen_visualscores)
    gen_r = lambda shape: 2
    pop.set_parameter_generator('r', gen_r , 'population')
    gen_f0 = lambda shape: np.random.randint(14, 18+1, size=shape)
    pop.set_parameter_generator('f0', gen_f0, 'person')

    #symptom score no error
    gen_symptomscores = lambda f,a,s0: np.maximum(a*f+s0, SMIN)
    pop.set_score_generator('symptomscore_noerror', gen_symptomscores)
    gen_a = lambda shape: 1
    pop.set_parameter_generator('a', gen_a, 'population')
    gen_s0 = lambda shape: np.random.normal(6, 2, size=shape)
    pop.set_parameter_generator('s0', gen_s0, 'person')

    #symptom error
    gen_error_mult = lambda s,C: s*C
    pop.set_score_generator('symptomscore', gen_error_mult)
    gen_C = lambda shape: 1 + np.random.choice([-1, 1], size=shape)*(np.random.randint(0, 30, size=shape)/100)
    pop.set_parameter_generator('C', gen_C, 'day')
    
    return pop

def get_prevsim_pop(size=10):
    pop = Population(size, 'previous simulation - basic pop')
    
    #visual score
    gen_visualscores = lambda t,r,f0: np.maximum(-r*t+f0, FMIN)
    pop.set_score_generator('visualscore', gen_visualscores)
    gen_r = lambda shape: 1
    pop.set_parameter_generator('r', gen_r , 'population')
    gen_f0 = lambda shape: random.normal(*uniform_to_normal((14, 18)), shape)
    pop.set_parameter_generator('f0', gen_f0, 'person')

    #symptom score no error
    gen_symptomscores = lambda f,a: a*(f-FMIN)
    pop.set_score_generator('symptomscore_noerror', gen_symptomscores)
    gen_a = lambda shape: random.choice([1, 2, 3], shape)
    pop.set_parameter_generator('a', gen_a, 'person')

    #symptom error
    gen_error_mult = lambda s,C: s*C
    pop.set_score_generator('symptomscore', gen_error_mult)
    gen_C = lambda shape: random.normal(*uniform_to_normal((0.8, 1.2), pmsigma=3), shape)
    pop.set_parameter_generator('C', gen_C, 'day')
    
    return pop

def get_linpop(size=10000):
    pop = get_default_pop(size)
    pop.title = 'linear population with multiplicative error'
    
    #symptom error
    gen_error_mult = lambda s,C: s*C
    pop.set_score_generator('symptomscore', gen_error_mult)
    gen_C = lambda shape: random.normal(*uniform_to_normal((0.7, 1.3), pmsigma=3), size=shape)
    pop.set_parameter_generator('C', gen_C, 'day')
    
    return pop

def get_exppop(size=10000):
    pop = get_linpop(size)
    pop.title = 'exponential population with multiplicative error'
    
    #symptom score no error
    gen_symptomscores = lambda f,a,f0,B: a*(f0-FMIN)*(B**f-B**FMIN)/(B**f0-B**FMIN)
    pop.set_score_generator('symptomscore_noerror', gen_symptomscores)
    gen_B = lambda shape: np.full(shape=shape, fill_value=1.5)
    pop.set_parameter_generator('B', gen_B, 'population')
    
    return pop

In [685]:
linpop = get_linpop(10)
linpop2 = linpop.copy()
linpop3 = linpop.copy(additionaltitle='(no error)')

linpop3.set_parameter_generator('C', lambda shap: 1, 'population')

linpop.generate()
linpop2.generate()
linpop3.generate()

nplots=3
fig, axes = plt.subplots(ncols=nplots, figsize=plt.figaspect(1/nplots)*1.25)
ndays = 15
linpop.plot(axes[0], ndays=ndays)
linpop2.plot(axes[1], ndays=ndays)
linpop3.plot(axes[2], ndays=ndays)

plt.show()

ValueError: The function for parameter generation should only have 'shape' as an argument. 
Currently, it's arguments are: 
'shap'

# Study

In [617]:
class Study:
    
    def __init__(self, title='', sample_args=[7, 14, 28], smilescore='symptomscore'):
        '''
        Sample args that are integers are interpreted as a fixed day
        Sample args between 0 and 1 (exclusive) are interpreted as a percentage with which to use SMILE, based on the first fixed day
        Sample args that do not fit the above criteria are ignored
        smilescore determines which score the % milestones will be based on (can be smilescore, visualscore, or smilescore_noerror)
        '''
        
        self.title=title
        self.smilescore=smilescore
        
        sample_args = np.array(sample_args)
        self.fixed_days = sample_args[sample_args == sample_args.astype(int)].astype(int)
        self.milestones = sample_args[(0 < sample_args) | (sample_args < 1)]
        
        if self.fixed_days.size==0: 
            raise Exception("No fixed days in sample_args, which were {}".format(sample_args))
        
        if np.max(self.fixed_days) >= NDAYS:
            raise Exception("There is a fixed sample day in {} that is later than the LASTVISIT of {}".format(self.fixed_days, LASTVISIT))
        if np.max(self.fixed_days) > LASTVISIT:
            warn("There is a fixed sample day in {} that is later than the LASTVISIT of {}".format(self.fixed_days, LASTVISIT))
        if np.min(self.fixed_days) < FIRSTVISIT:
            warn("There is a fixed sample day in {} that is earlier than the FIRSTVISIT of {}".format(self.fixed_days, FIRSTVISIT))
        self.sample_days=np.array(sample_days)
        
    def sample(population, filter=True): #TODO implement with milestones
        if filter: population = deepcopy(population).filter()
                 
        if self.smilescore == 'visualscore':
            smilescores = population.visualscores
        elif self.smilescore == 'symptomscore':
            smilescores = population.symptomscores
        elif self.smilescore == 'symptomscore_noerror':
            smilescores = population.symptomscores_noerror
                 
        index = np.min(self.fixed_days) #day which SMILEs are based on
        index_vals = smilescores[:, index].reshape(-1, 1) #reshape to be vertical
        smile_vals = index_vals*self.milestones #each row is a person, each column is a milestone
        
        firstdays = np.empty_like(smile_vals)
        for milestone_col in range(smile_vals.shape[1]):
            milestone_vals = smile_vals[:,col].reshape(-1, 1)
            firstdays[:,milestone_col] = np.argmax(smilescores <= milestone_vals, axis=1).astype(int)
        #values of 0 in firstdays matrix might be either 'milestone at day 0' or 'milestone not reached' due to how np.argmax works
        #we now replace those 'milestone not reached' with 'NaN' values
        scores_at_milestonedays = np.take_along_axis(smilescores, firsdays, 1)
        scores_at_milestonedays = np.where(scores_at_milestonedays <= milestone_vals, firsdays, np.nan)
        
        #we can merge milestonedays with fixed_days since they both represent day indices
        sampledays = np.concatenate([scores_at_milestonedays, self.fixed_days], axis=1)
        sampledays = np.sort(sampledays) #day indices are sorted for each person
        
        samplevals = np.take_along_axis(population.symptomscores, sampledays, axis=1) #will throw error if some milestone not reached TODO fix
                
        return samplevals
                 
    #TODO analyze method
                 
    #TODO compare_analysis classmethod

In [618]:
#TODO not only first dip, but multiple consecutive days
#TODO explain that reshape(-1,1) makes rows into columns

In [619]:
symp = linpop.symptomscores[:,:15]
smile_vals = linpop.symptomscores[:, 0].reshape(-1, 1)*np.array([1, 0.5, 0.1, -1])
arr = np.empty_like(smile_vals)
for col in range(smile_vals.shape[1]):
    milestone = smile_vals[:,col].reshape(-1,1)
    where_symp_leq_milestone = (symp <= milestone)
    arr[:,col] = np.argmax(where_symp_leq_milestone, axis=1)
brr = np.take_along_axis(symp, arr.astype(int), 1)
crr = brr <= smile_vals
crr = (np.where(crr, arr, np.nan))
drr = np.array([1, 2])
drr = (np.tile(drr, (10,1)))
err = (np.concatenate([crr, drr], axis=1))
print(err)
print(np.sort(err).astype(int))
np.take_along_axis(symp, np.sort(err).astype(int), 1)

[[ 0.  3.  9. nan  1.  2.]
 [ 0.  6. 10. nan  1.  2.]
 [ 0.  4.  9. nan  1.  2.]
 [ 0.  5. 10. nan  1.  2.]
 [ 0.  5.  9. nan  1.  2.]
 [ 0.  6.  9. nan  1.  2.]
 [ 0.  5.  9. nan  1.  2.]
 [ 0.  4.  9. nan  1.  2.]
 [ 0.  5. 10. nan  1.  2.]
 [ 0.  6.  9. nan  1.  2.]]
[[          0           1           2           3           9 -2147483648]
 [          0           1           2           6          10 -2147483648]
 [          0           1           2           4           9 -2147483648]
 [          0           1           2           5          10 -2147483648]
 [          0           1           2           5           9 -2147483648]
 [          0           1           2           6           9 -2147483648]
 [          0           1           2           5           9 -2147483648]
 [          0           1           2           4           9 -2147483648]
 [          0           1           2           5          10 -2147483648]
 [          0           1           2           6     

IndexError: index -2147483648 is out of bounds for axis 1 with size 15